In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
import os

In [ ]:
from tensorflow.keras.datasets import cifar10

(train_images, train_labels),(test_images, test_labels) = cifar10.load_data()
print(train_images.shape)

In [ ]:
NAMES = np.array(['airplane','automobile','bird','cat','deer','dog','frog','horse','ship','truck'])
print(train_labels[:10])

In [ ]:
import matplotlib.pyplot as plt
import cv2
%matplotlib inline

def show_images(images, labels, ncols=8):
    figure, axs = plt.subplots(figsize=(22,6),nrows=1, ncols=ncols)
    for i in range(ncols):
        axs[i].imshow(images[i])
        label = labels[i].squeeze()
        axs[i].set_title(NAMES[int(label)])
        
show_images(train_images[:8],train_labels[:8],ncols=8)
show_images(train_images[8:16], train_labels[8:16], ncols=8)

In [ ]:
def get_preprocessed_data(images, labels):
    images = np.array(images/255.0, dtype = np.float32)
    labels = np.array(labels, dtype=np.float32)
    
    return images, labels

train_images, train_labels = get_preprocessed_data(train_images, train_labels)
test_images, test_labels = get_preprocessed_data(test_images, test_labels)

In [ ]:
train_images[0,:,:,:]

In [ ]:
train_labels = train_labels.squeeze()
test_labels = test_labels.squeeze()

In [ ]:
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Input, Dense, Conv2D, Dropout, Flatten, Activation, MaxPooling2D, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint, LearningRateScheduler

IMAGE_SIZE=32

input_tensor = Input(shape=(IMAGE_SIZE,IMAGE_SIZE,3))
x = Conv2D(filters=32, kernel_size=(5,5),padding='valid',activation='relu')(input_tensor)
x = Conv2D(filters=32, kernel_size=(3,3),padding='same',activation='relu')(x)
x = MaxPooling2D(pool_size=(2,2))(x)

x = Conv2D(filters=64, kernel_size=(3,3),padding='same',activation='relu')(x)
x = Conv2D(filters=64, kernel_size=(3,3),padding='same')(x)
x = Activation('relu')(x)
x = MaxPooling2D(pool_size=2)(x)

x = Conv2D(filters=128, kernel_size=(3,3),padding='same',activation='relu')(x)
x = Conv2D(filters=128, kernel_size=(3,3),padding='same',activation='relu')(x)
x = MaxPooling2D(pool_size=2)(x)

x = Flatten(name='flatten')(x)
x = Dropout(rate=0.5)(x)
x = Dense(300, activation='relu',name='fc1')(x)
x = Dropout(rate=0.3)(x)
output = Dense(10, activation='softmax',name='output')(x)

model = Model(inputs = input_tensor, outputs=output)
model.summary()

In [ ]:
model.compile(optimizer=Adam(),loss='sparse_categorical_crossentropy',metrics=['accuracy'])
history = model.fit(x=train_images,y=train_labels, batch_size=64,epochs=30,validation_split=0.15)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

def show_history(history):
    plt.figure(figsize=(6,6))
    plt.yticks(np.arange(0,1,0.05))
    plt.plot(history.history['accuracy'],label = 'train')
    plt.plot(history.history['val_accuracy'], label='valid')
    plt.legend()
    
show_history(history)

In [ ]:
model.evaluate(test_images, test_labels)

In [ ]:
#4차원
preds = model.predict(np.expand_dims(test_images[0],axis=0))
print(preds.shape)
print(preds)

In [ ]:
preds = model.predict(test_images[:32], batch_size=32)
print(preds.shape)
print(preds)

In [ ]:
predicted_class = np.argmax(preds, axis=1)
print(predicted_class)

In [ ]:
show_images(test_images[:8],predicted_class[:8],ncols=8)
show_images(test_images[8:16], predicted_class[8:16], ncols=8)

In [ ]:
import pandas as pd
import numpy as np
import os 
numbers = np.random.normal(loc=0.0, scale=1, size=[100,100])
print(numbers)
print(numbers.mean())
print(numbers.std())
print(numbers.sum())

In [ ]:
fan_in = 20
fan_out = 15
scale_value = np.sqrt(2/(fan_in + fan_out))
print('scale:',scale_value)
weights = np.random.normal(loc=0.0, scale= scale_value, size=[100,100])
print(weights)
print(weights.mean(), weights.std(), weights.sum())

In [ ]:
fan_in = 10
fan_out = 8
limit = np.sqrt(6/(fan_in+fan_out))
print(limit)
weights = np.random.uniform(-1*limit, limit, size=[100,100])
print(weights)
print(weights.mean(), weights.std(), weights.sum())

In [ ]:
fan_in = 10
scale_value = np.sqrt(2/(fan_in))
print(scale_value)
weights = np.random.normal(loc=0.0, scale = scale_value, size=(100,100))
print(weights)
print(weights.mean(), weights.std(), weights.sum())

In [ ]:
fan_in = 10
limit = np.sqrt(6/fan_in)
weights = np.random.uniform(limit*-1, limit, size=(100,100))
print(weights)
print(weights.mean(), weights.std(), weights.sum())

In [ ]:
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical

def get_preprocessed_data(images, labels):
    
    # 학습과 테스트 이미지 array를 0~1 사이값으로 scale 및 float32 형 변형. 
    images = np.array(images/255.0, dtype=np.float32)
    labels = np.array(labels, dtype=np.float32)
    labels = labels.squeeze()
    
    return images, labels

# 0 ~ 1사이값 float32로 변경하는 함수 호출 한 뒤 OHE 적용 
def get_preprocessed_ohe(images, labels):
    images, labels = get_preprocessed_data(images, labels)
    # OHE 적용 
    oh_labels = to_categorical(labels)
    return images, oh_labels

(train_images, train_labels), (test_images, test_labels) = cifar10.load_data()

train_images, train_oh_labels = get_preprocessed_ohe(train_images, train_labels)
test_images, test_oh_labels = get_preprocessed_ohe(test_images, test_labels)
print(train_images.shape, train_oh_labels.shape, test_images.shape, test_oh_labels.shape)

In [ ]:
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Input, Dense , Conv2D , Dropout , Flatten , Activation, MaxPooling2D , GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam , RMSprop 
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.callbacks import ReduceLROnPlateau , EarlyStopping , ModelCheckpoint , LearningRateScheduler

input_tensor = Input(shape=(IMAGE_SIZE, IMAGE_SIZE, 3))

#x = Conv2D(filters=32, kernel_size=(5, 5), padding='valid', activation='relu')(input_tensor)
x = Conv2D(filters=32, kernel_size=(3, 3), padding='same', activation='relu', kernel_initializer='he_normal')(input_tensor)
x = Conv2D(filters=32, kernel_size=(3, 3), padding='same', activation='relu', kernel_initializer='he_normal')(x)
x = MaxPooling2D(pool_size=(2, 2))(x)

x = Conv2D(filters=64, kernel_size=3, padding='same', activation='relu', kernel_initializer='he_normal')(x)
x = Conv2D(filters=64, kernel_size=3, padding='same', kernel_initializer='he_normal')(x)
x = Activation('relu')(x)
x = MaxPooling2D(pool_size=2)(x)

x = Conv2D(filters=128, kernel_size=3, padding='same', activation='relu', kernel_initializer='he_normal')(x)
x = Conv2D(filters=128, kernel_size=3, padding='same', activation='relu', kernel_initializer='he_normal')(x)
x = MaxPooling2D(pool_size=2)(x)

# cifar10의 클래스가 10개 이므로 마지막 classification의 Dense layer units갯수는 10
x = Flatten(name='flatten')(x)
x = Dropout(rate=0.5)(x)
x = Dense(300, activation='relu', name='fc1')(x)
x = Dropout(rate=0.3)(x)
output = Dense(10, activation='softmax', name='output')(x)

model = Model(inputs=input_tensor, outputs=output)

model.summary()

In [ ]:
import tensorflow as tf
import numpy as np
import random as python_random

np.random.seed(2021)
python_random.seed(2021)
tf.random.set_seed(2021)

In [ ]:
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Input, Dense , Conv2D , Dropout , Flatten , Activation, MaxPooling2D , GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam , RMSprop 
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.callbacks import ReduceLROnPlateau , EarlyStopping , ModelCheckpoint , LearningRateScheduler

IMAGE_SIZE=32
input_tensor = Input(shape=(IMAGE_SIZE, IMAGE_SIZE, 3))

#x = Conv2D(filters=32, kernel_size=(5, 5), padding='valid', activation='relu')(input_tensor)
x = Conv2D(filters=32, kernel_size=(3, 3), padding='same')(input_tensor)
x = BatchNormalization()(x)
x = Activation('relu')(x)

x = Conv2D(filters=32, kernel_size=(3, 3), padding='same')(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = MaxPooling2D(pool_size=(2, 2))(x)

x = Conv2D(filters=64, kernel_size=3, padding='same')(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)

x = Conv2D(filters=64, kernel_size=3, padding='same')(x)
x = Activation('relu')(x)
x = Activation('relu')(x)
x = MaxPooling2D(pool_size=2)(x)

x = Conv2D(filters=128, kernel_size=3, padding='same')(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)

x = Conv2D(filters=128, kernel_size=3, padding='same')(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = MaxPooling2D(pool_size=2)(x)

# cifar10의 클래스가 10개 이므로 마지막 classification의 Dense layer units갯수는 10
x = Flatten(name='flatten')(x)
x = Dropout(rate=0.5)(x)
x = Dense(300, activation='relu', name='fc1')(x)
x = Dropout(rate=0.3)(x)
output = Dense(10, activation='softmax', name='output')(x)

model = Model(inputs=input_tensor, outputs=output)
model.summary()

In [ ]:
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
history = model.fit(x=train_images,y=train_oh_labels,batch_size=64,epochs=30,validation_split=0.15)

In [ ]:
model.evaluate(test_images, test_oh_labels)

In [ ]:
preds = model.predict(np.expand_dims(test_images[0],axis=0))
predicted_class = np.argmax(preds,axis=1)
print(predicted_class)

In [ ]:
import tensorflow as tf
import numpy as np
import random as python_random
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

# seed 를 설정해서 학습시마다 동일한 결과 유도. 불행히도 의도한 대로 동작하지 않음. 
def set_random_seed(seed_value):
    np.random.seed(seed_value)
    python_random.seed(seed_value)
    tf.random.set_seed(seed_value)

# 0 ~ 1사이값의 float32로 변경하는 함수
def get_preprocessed_data(images, labels):
    
    # 학습과 테스트 이미지 array를 0~1 사이값으로 scale 및 float32 형 변형. 
    images = np.array(images/255.0, dtype=np.float32)
    labels = np.array(labels, dtype=np.float32)
    
    return images, labels

# 0 ~ 1사이값 float32로 변경하는 함수 호출 한 뒤 OHE 적용 
def get_preprocessed_ohe(images, labels):
    images, labels = get_preprocessed_data(images, labels)
    # OHE 적용 
    oh_labels = to_categorical(labels)
    return images, oh_labels

# 학습/검증/테스트 데이터 세트에 전처리 및 OHE 적용한 뒤 반환 
def get_train_valid_test_set(train_images, train_labels, test_images, test_labels, valid_size=0.15, random_state=2021):
    # 학습 및 테스트 데이터 세트를  0 ~ 1사이값 float32로 변경 및 OHE 적용. 
    train_images, train_oh_labels = get_preprocessed_ohe(train_images, train_labels)
    test_images, test_oh_labels = get_preprocessed_ohe(test_images, test_labels)
    
    # 학습 데이터를 검증 데이터 세트로 다시 분리
    tr_images, val_images, tr_oh_labels, val_oh_labels = train_test_split(train_images, train_oh_labels, test_size=valid_size, random_state=random_state)
    
    return (tr_images, tr_oh_labels), (val_images, val_oh_labels), (test_images, test_oh_labels ) 


In [ ]:
from tensorflow.keras.datasets import cifar10

set_random_seed(2021)
(train_images, train_labels), (test_images, test_labels) = cifar10.load_data()
(tr_images, tr_oh_labels), (val_images, val_oh_labels), (test_images, test_oh_labels) = get_train_valid_test_set(train_images, train_labels, test_images, test_labels, valid_size=0.15, random_state=2021)

In [ ]:
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Input,Dense,Conv2D,Dropout,Flatten,Activation,MaxPooling2D,GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint, LearningRateScheduler

def create_model():
    
    input_tensor = Input(shape=(IMAGE_SIZE,IMAGE_SIZE,3))
    
    x = Conv2D(filters=32,kernel_size=3, padding='same')(input_tensor)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    
    x = Conv2D(filters=32,kernel_size=3,padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = MaxPooling2D(pool_size=(2,2))(x)
    
    x = Conv2D(filters=64, kernel_size=3, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = Conv2D(filters=64, kernel_size=3, padding='same')(x)
    x = Activation('relu')(x)
    x = Activation('relu')(x)
    x = MaxPooling2D(pool_size=2)(x)
    
    x = Conv2D(filters=128, kernel_size=3, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = Conv2D(filters=128, kernel_size=3, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = MaxPooling2D(pool_size=2)(x)
    
    x = Flatten(name = 'flatten')(x)
    x = Dropout(rate=0.5)(x)
    x = Dense(300, activation='relu',name='fc1')(x)
    x = Dropout(rate=0.3)(x)
    output = Dense(10,activation='softmax',name='output')(x)
    
    
    model = Model(inputs = input_tensor, outputs=output)
    model.summary()
    
    return model


In [ ]:
model = create_model()
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

noshuffle_history = model.fit(x=tr_images, y=tr_oh_labels, batch_size=64, epochs=30,shuffle=False,validation_data=(val_images, val_oh_labels))
evaluation_result = model.evaluate(test_images, test_oh_labels, batch_size=64)

print(evaluation_result)

tf.keras.backend.clear_session()

In [ ]:
model=create_model()
model.compile(optimizer=Adam(), loss='categorical_crossentropy',metrics=['accuracy'])
shuffle_history = model.fit(x=tr_images, y=tr_oh_labels, batch_size=64, epochs=30, shuffle=True, 
                    validation_data=(val_images, val_oh_labels))
evaluation_result = model.evaluate(test_images, test_oh_labels, batch_size=64)
print('#### 테스트 세트로 evaluation 결과 :', evaluation_result)

tf.keras.backend.clear_session()

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

def show_history_shuffle(noshuffle_history, shuffle_history):
    figure, axs = plt.subplots(nrows=1, ncols=2, figsize=(16,4))
    
    axs[0].plot(noshuffle_history.history['val_accuracy'],label='no shuffle acc')
    axs[0].plot(shuffle_history.history['val_accuracy'],label='shuffle acc')
    
    axs[1].plot(noshuffle_history.history['val_loss'],label='no shuffle loss')
    axs[1].plot(shuffle_history.history['val_loss'],label='shuffle loss')
    axs[0].legend()
    axs[1].legend()
    
show_history_shuffle(noshuffle_history, shuffle_history)

In [ ]:
b_sizes=[32,64,256,512]
histories = []
evaluations = []

for b_size in b_sizes:
    model = create_model()
    model.compile(optimizer=Adam(),loss='categorical_crossentropy',metrics=['accuracy'])
    history = model.fit(x=tr_images, y=tr_oh_labels, batch_size=b_size, epochs=30, shuffle=True,validation_data=(val_images, val_oh_labels))
    histories.append(history)
    
    evaluation_result=model.evaluate(test_images,test_oh_labels, batch_size=b_size)
    evaluations.append(evaluation_result)
    
    tf.keras.backend.clear_session()

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

def show_history_batch(histories):
    figure, axs = plt.subplots(nrows=1, ncols=2, figsize=(16, 4))  
    # batch 크기별 validation accuracy 비교 
    axs[0].plot(histories[0].history['val_accuracy'], label='batch 32 acc')
    axs[0].plot(histories[1].history['val_accuracy'], label='batch 64 acc')
    axs[0].plot(histories[2].history['val_accuracy'], label='batch 256 acc')
    axs[0].plot(histories[3].history['val_accuracy'], label='batch 512 acc')
    
    # batch 크기별 validation loss 비교
    axs[1].plot(histories[0].history['val_loss'], label='batch 32 loss')
    axs[1].plot(histories[1].history['val_loss'], label='batch 64 loss')
    axs[1].plot(histories[2].history['val_loss'], label='batch 256 loss')
    axs[1].plot(histories[3].history['val_loss'], label='batch 512 loss')
    
    axs[0].legend()
    axs[1].legend()

show_history_batch(histories)